# Trigonometrie

## Medieninformatik

### 3D-Rendering
Dreiecke sind recht einfach geometrische Formen, die auch gut bekannt und erforscht sind. Aus der Geodäsie ist bekannt, dass Oberflächen sehr gut durch Dreiecke approximiert werden können. Aus diesem Grund bedient man sich auch oft im Bereich des 3D-Renderings (Animationsfilme, Videospiele, CAD, ...) einer Approximation der darzustellenden Oberflächen durch Dreiecke. Da die Trigonometrie das Maß der Dreiecke ist, soll das Rendern von 3D-Objekten hier als Motivation dienen sich mit Dreiecken und den trigonometrischen Funktionen auseinanderzusetzen. 

Bitte beachten Sie in diesem Beispiel, dass Python 3.8 vorausgesetzt wird. Zwar ist Python 3.9 bereits länger verfügbar, für die Bibliothek Open3D gibt es aber noch keine kompatible Adaption. Sollten Sie der Anleitung gefolgt sein und bereit Anaconda installiert haben, haben Sie Python 3.8. Sollten Sie Anaconda erst später installieren oder Python direkt installieren wollen, müssen Sie den Hinweis beachten. 

Und hier genutzte Beispiel in einem Blogeintrag auf [Towards Data Science](https://towardsdatascience.com/5-step-guide-to-generate-3d-meshes-from-point-clouds-with-python-36bad397d8ba) entdeckt. 

In [1]:
# Führen Sie dieses Kommando aus um Open3D zu installieren:
# !pip3 install open3d
# oder alternativ
# !pip install open3d

In [2]:
# Required imports
import open3d as o3d
import numpy as np


# Function to calculate different level of details
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods


# Create paths and load data
input_path="./"
output_path="./"
dataname="sample_w_normals.xyz"
point_cloud= np.loadtxt(input_path+dataname,skiprows=1)

# Load point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

# Radius determination
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

# Create mesh using the Ball-Pivoting Algorithm 
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

# Calculate level of details
my_lods = lod_mesh_export(bpa_mesh, [1,2,10,50,100,1000,10000,100000], ".ply", output_path)

generation of 100000 LoD successful


In [11]:
o3d.visualization.draw_geometries([my_lods[100000]])

In [14]:
points = bpa_mesh.sample_points_uniformly(number_of_points=100000)

In [15]:
o3d.visualization.draw_geometries([points])